In [4]:
import rasterio

with rasterio.open('../../data/source_data/with_s2_/jiri_1.tif') as src:
    old_data = src.read()
    
with rasterio.open('../../data/source_data/with_s2/jiri_1.tif') as src:
    new_data = src.read()

In [7]:
import numpy as np

old_data = np.transpose(old_data.reshape(12, 9, 3000, 3000),(1,0,2,3))

In [8]:
import numpy as np

new_data = new_data.reshape(10, 12, 3600, 3600)

In [9]:
old_data.shape, new_data.shape

((9, 12, 3000, 3000), (10, 12, 3600, 3600))

In [13]:
np.mean(old_data, axis=(1,2,3))

array([ 168.9760456 ,  400.17113819,  351.36498038, 2873.97721556,
       2071.49668991, 3454.45096676, 3917.9825862 , 3016.33497763,
       2145.04939846])

In [14]:
np.mean(new_data, axis=(1,2,3))

array([ 169.03055303,  400.20804956,  351.43169117, 2873.63905477,
       2127.45015148, 3603.91675592, 4080.6677801 , 4375.49492688,
       3146.54678412, 2190.47362054])

In [15]:
import numpy as np
from scipy.stats import trim_mean

def trimmed_mean_nd(data, proportiontocut, axis):
    """
    다차원 배열에서 특정 축을 따라 절사 평균을 계산하는 함수.
    
    :param data: 입력 데이터 (ndarray)
    :param proportiontocut: 절사 비율 (예: 0.2 → 상하위 20% 제거)
    :param axis: 평균을 계산할 축 (튜플 형태로 다중 축 가능)
    :return: 절사 평균이 적용된 결과 배열
    """
    # 축을 펼쳐서 계산한 뒤 다시 원래 형태로 변환
    def trimmed_mean_1d(arr):
        return trim_mean(arr, proportiontocut)

    return np.apply_along_axis(trimmed_mean_1d, axis, data)

In [17]:
# 절사 평균 적용
old_avg = trimmed_mean_nd(old_data, proportiontocut=0.2, axis=(2,3))
new_avg = trimmed_mean_nd(new_data, proportiontocut=0.2, axis=(2,3))

TypeError: 'tuple' object cannot be interpreted as an integer

In [11]:
import numpy as np
import rasterio
import os
from scipy.stats import trim_mean

ratio = {'jiri_1': [[0.9741672 , 0.96803265, 0.97050439, 0.97428304, 0.9807188 ,
         0.97300037, 0.91671335, 1.01871054, 0.98248265, 0.94867588,
         0.9615212 , 0.99800107],
        [0.98222879, 0.96131036, 0.97794913, 0.97499468, 0.95701381,
         0.97149606, 0.94566925, 0.90067255, 0.94602027, 0.94507367,
         0.99695092, 0.99393134],
        [0.98344452, 0.96407934, 0.9776159 , 0.97475823, 0.9571236 ,
         0.97183663, 0.94940729, 0.90876641, 0.95044012, 0.94852773,
         0.99980358, 0.98911588],
        [0.98160618, 0.96756826, 0.97414382, 0.97233954, 0.9593882 ,
         0.97046858, 0.9531524 , 0.90899106, 0.9543194 , 0.94470305,
         0.98803374, 0.98723817],
        [0.94745622, 0.97195097, 0.94145404, 0.96603916, 0.97387321,
         0.95012486, 0.97340187, 0.92605678, 0.94855845, 0.94431985,
         0.9376906 , 0.98385372],
        [0.95043139, 0.98456966, 0.94349072, 0.97730798, 0.97822162,
         0.94890065, 0.97930823, 1.03859313, 0.96367152, 0.96822611,
         0.94059858, 1.01554205]],
 'jiri_2': [[0.97552935, 0.96802737, 0.96963225, 0.97304612, 0.97584902,
         0.97026942, 0.91510461, 1.01705121, 0.98037721, 0.94633571,
         0.95534227, 0.9952709 ],
        [0.98800574, 0.96951764, 0.97917915, 0.97656929, 0.9590074 ,
         0.97399949, 0.94591461, 0.91445399, 0.9495055 , 0.94605861,
         0.99564354, 0.98877293],
        [0.98690608, 0.97020874, 0.9771879 , 0.97443172, 0.95771995,
         0.97188737, 0.94937304, 0.91166062, 0.94473262, 0.94665691,
         0.99731786, 0.98391137],
        [0.98378039, 0.97217083, 0.97360315, 0.97031506, 0.9595955 ,
         0.97030356, 0.95451566, 0.908026  , 0.95090315, 0.944803  ,
         0.98971569, 0.98279056],
        [0.94951819, 0.97677474, 0.94175399, 0.96860167, 0.97156371,
         0.95075624, 0.97398878, 0.96448269, 0.94919178, 0.9441581 ,
         0.93635292, 0.98411633],
        [0.94587825, 0.98134274, 0.93887724, 0.97329765, 0.97322454,
         0.94765743, 0.97835274, 1.08501257, 0.96229674, 0.96290331,
         0.93717946, 1.00844349]],
 'sobaek': [[0.97360352, 0.96791537, 0.96797445, 0.97311546, 0.96822336,
         0.96730015, 0.93179224, 1.00941756, 0.98675074, 0.9362247 ,
         0.95674153, 0.9872868 ],
        [0.98885844, 0.96384831, 0.97544901, 0.97328658, 0.96487016,
         0.96808337, 0.94452797, 0.89575521, 0.9595786 , 0.93418757,
         0.98828003, 0.96492221],
        [0.9889863 , 0.96383738, 0.97213748, 0.970976  , 0.96346772,
         0.96806564, 0.94379134, 0.90169493, 0.95869145, 0.93613007,
         0.9929039 , 0.95985244],
        [0.98576013, 0.96474744, 0.96841962, 0.96843976, 0.96517876,
         0.96627842, 0.94577227, 0.90299585, 0.95802261, 0.93257272,
         0.98645901, 0.96270556],
        [0.94570988, 0.97048514, 0.93396276, 0.96958479, 0.96627689,
         0.94570838, 0.96562306, 0.89469907, 0.95444608, 0.93380812,
         0.93647028, 0.97400437],
        [0.95175413, 0.97906008, 0.94346049, 0.98164556, 0.96728445,
         0.94345497, 0.9715978 , 0.96774523, 0.96739353, 0.95725168,
         0.94577468, 1.00023305]]}

for key in ratio.keys():
    ratio[key] = np.array(ratio[key])


for r in ['jiri_1', 'jiri_2', 'sobaek']:
    with rasterio.open(f'../../data/source_data/with_s2/{r}.tif') as src:
        new_data = src.read()  # (12, 3600, 3600)
        new_data = new_data.reshape(10, 12, 3600, 3600)  # (10, 12, 3600, 3600)

    # Scaling 적용
    new_data[4:] = new_data[4:] * ratio[r][:, :, np.newaxis, np.newaxis]  # Broadcasting 적용 (6,12) → (6,12,3600,3600)

    # 소수점 첫째 자리에서 반올림 후 uint16 변환
    new_data = np.round(new_data).astype(np.uint16)
    new_data = new_data.reshape(120,3600,3600)
    
    # 수정된 데이터 저장
    output_path = f'../../data/source_data/with_s2_revised/{r}.tif'
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=new_data.shape[1],
        width=new_data.shape[2],
        count=new_data.shape[0],
        dtype=np.uint16,
        crs=src.crs,
        transform=src.transform
    ) as dst:
        dst.write(new_data)

print("Processing complete.")


In [12]:


for key in ratio.keys():
    ratio[key] = np.array(ratio[key])

In [14]:
ratio['jiri_1']

array([[0.9741672 , 0.96803265, 0.97050439, 0.97428304, 0.9807188 ,
        0.97300037, 0.91671335, 1.01871054, 0.98248265, 0.94867588,
        0.9615212 , 0.99800107],
       [0.98222879, 0.96131036, 0.97794913, 0.97499468, 0.95701381,
        0.97149606, 0.94566925, 0.90067255, 0.94602027, 0.94507367,
        0.99695092, 0.99393134],
       [0.98344452, 0.96407934, 0.9776159 , 0.97475823, 0.9571236 ,
        0.97183663, 0.94940729, 0.90876641, 0.95044012, 0.94852773,
        0.99980358, 0.98911588],
       [0.98160618, 0.96756826, 0.97414382, 0.97233954, 0.9593882 ,
        0.97046858, 0.9531524 , 0.90899106, 0.9543194 , 0.94470305,
        0.98803374, 0.98723817],
       [0.94745622, 0.97195097, 0.94145404, 0.96603916, 0.97387321,
        0.95012486, 0.97340187, 0.92605678, 0.94855845, 0.94431985,
        0.9376906 , 0.98385372],
       [0.95043139, 0.98456966, 0.94349072, 0.97730798, 0.97822162,
        0.94890065, 0.97930823, 1.03859313, 0.96367152, 0.96822611,
        0.94059858,

In [17]:
import rasterio
import numpy as np
import os
from scipy.stats import trim_mean


for r in ['jiri_1', 'jiri_2', 'sobaek']:
    # 새로운 데이터 읽기 및 평균 계산
    with rasterio.open(f'../../data/source_data/with_s2/{r}.tif') as src:
        new_data = src.read()  # (12, 3600, 3600)
        new_data = new_data.reshape(10, 12, 3600, 3600)  # (10, 12, 3600, 3600)

    # Scaling 적용
    new_data[4:] = new_data[4:] * ratio[r][:, :, np.newaxis, np.newaxis]  # Broadcasting 적용 (6,12) → (6,12,3600,3600)

    # 소수점 첫째 자리에서 반올림 후 uint16 변환
    new_data = np.round(new_data).astype(np.uint16)
    new_data = new_data.reshape(120,3600,3600)
    
    # 수정된 데이터 저장
    output_path = f'../../data/source_data/with_s2_revised/{r}.tif'
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=new_data.shape[1],
        width=new_data.shape[2],
        count=new_data.shape[0],
        dtype=np.uint16,
        crs=src.crs,
        transform=src.transform
    ) as dst:
        dst.write(new_data)

print("Processing complete.")


Processing complete.


In [15]:
ratio['jiri_1']

array([[0.9741672 , 0.96803265, 0.97050439, 0.97428304, 0.9807188 ,
        0.97300037, 0.91671335, 1.01871054, 0.98248265, 0.94867588,
        0.9615212 , 0.99800107],
       [0.98222879, 0.96131036, 0.97794913, 0.97499468, 0.95701381,
        0.97149606, 0.94566925, 0.90067255, 0.94602027, 0.94507367,
        0.99695092, 0.99393134],
       [0.98344452, 0.96407934, 0.9776159 , 0.97475823, 0.9571236 ,
        0.97183663, 0.94940729, 0.90876641, 0.95044012, 0.94852773,
        0.99980358, 0.98911588],
       [0.98160618, 0.96756826, 0.97414382, 0.97233954, 0.9593882 ,
        0.97046858, 0.9531524 , 0.90899106, 0.9543194 , 0.94470305,
        0.98803374, 0.98723817],
       [0.94745622, 0.97195097, 0.94145404, 0.96603916, 0.97387321,
        0.95012486, 0.97340187, 0.92605678, 0.94855845, 0.94431985,
        0.9376906 , 0.98385372],
       [0.95043139, 0.98456966, 0.94349072, 0.97730798, 0.97822162,
        0.94890065, 0.97930823, 1.03859313, 0.96367152, 0.96822611,
        0.94059858,